# ControlD API tool use

Claude tool use with ControlD API endpoints.

In [85]:
from anthropic import Anthropic
import requests
from typing import Dict, List, Optional
import os
from datetime import date
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Anthropic client with API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("Please set ANTHROPIC_API_KEY environment variable")
    
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME = "claude-3-5-sonnet-20241022"

In [86]:
def redact_sensitive_info(text: str, auth_token: str) -> str:
    """Redact sensitive information from output"""
    if not isinstance(text, str):
        text = str(text)
        
    # Redact full token
    if auth_token in text:
        text = text.replace(auth_token, "[REDACTED_TOKEN]")
    
    # Redact token in Bearer format
    text = text.replace(f"Bearer {auth_token}", "Bearer [REDACTED]")
    
    # Redact token in api.* format
    if "api." in text:
        text = text.replace(text[text.find("api."):text.find("api.") + 71], "[REDACTED_API_TOKEN]")
    
    return text

In [87]:
def get_controld_profiles(auth_token: str) -> Dict:
    """List all profiles"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        'https://api.controld.com/profiles',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

def create_controld_profile(auth_token: str, name: str, clone_profile_id: Optional[str] = None) -> Dict:
    """Create a new profile or clone an existing one"""
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}',
        'content-type': 'application/x-www-form-urlencoded'
    }
    
    data = {'name': name}
    if clone_profile_id:
        data['clone_profile_id'] = clone_profile_id
    
    # Debug logging with redaction
    print("\nRequest Details:")
    print(f"URL: https://api.controld.com/profiles")
    print(f"Method: POST")
    print(f"Headers: {redact_sensitive_info(str(headers), auth_token)}")
    print(f"Data: {data}")
    
    response = requests.post(
        'https://api.controld.com/profiles',
        headers=headers,
        data=data
    )
    
    # Debug logging with redaction
    print("\nResponse Details:")
    print(f"Status Code: {response.status_code}")
    print(f"Response Headers: {dict(response.headers)}")
    print(f"Response Body: {redact_sensitive_info(response.text, auth_token)}")
    
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"\nError: {str(e)}")
        return {"error": str(e)}

# Add this function with your other API functions
def get_controld_filters(auth_token: str, profile_id: str) -> Dict:
    """List all native filters for a specific profile
    
    Args:
        auth_token (str): ControlD API authentication token
        profile_id (str): Primary key (PK) of the profile
        
    Returns:
        Dict: JSON response containing filter information
    """
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}'
    }
    
    response = requests.get(
        f'https://api.controld.com/profiles/{profile_id}/filters',
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()

# Add this function with your other API functions
def modify_controld_filter(auth_token: str, profile_id: str, filter_name: str, status: int) -> Dict:
    """Modify a filter's status for a specific profile
    
    Args:
        auth_token (str): ControlD API authentication token
        profile_id (str): Primary key (PK) of the profile
        filter_name (str): Name of the filter to modify (PK from List Filters endpoint)
        status (int): Status to set (1 to enable, 0 to disable)
        
    Returns:
        Dict: JSON response confirming the modification
    """
    headers = {
        'accept': 'application/json',
        'authorization': f'Bearer {auth_token}',
        'content-type': 'application/x-www-form-urlencoded'
    }
    
    data = {
        'status': status
    }
    
    response = requests.put(
        f'https://api.controld.com/profiles/{profile_id}/filters/filter/{filter_name}',
        headers=headers,
        data=data
    )
    
    response.raise_for_status()
    return response.json()

In [88]:
tools = [
    {
        "name": "get_profiles",
        "description": "Retrieves all profiles associated with a ControlD account",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                }
            },
            "required": ["auth_token"]
        }
    },
    {
        "name": "create_profile",
        "description": "Creates a new ControlD profile or clones an existing one",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "name": {
                    "type": "string",
                    "description": "Name of the new profile"
                },
                "clone_profile_id": {
                    "type": "string",
                    "description": "The exact profile ID (PK) to clone from. Must be the full ID string, not just the profile name."
                }
            },
            "required": ["auth_token", "name"]
        }
    },
    {
        "name": "get_filters",
        "description": "Retrieves all native filters for a specific ControlD profile",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "profile_id": {
                    "type": "string",
                    "description": "Primary key (PK) of the profile to get filters for"
                }
            },
            "required": ["auth_token", "profile_id"]
        }
    },
    {
        "name": "modify_filter",
        "description": "Enables or disables a filter for a specific ControlD profile",
        "input_schema": {
            "type": "object",
            "properties": {
                "auth_token": {
                    "type": "string",
                    "description": "ControlD API authentication token"
                },
                "profile_id": {
                    "type": "string",
                    "description": "Primary key (PK) of the profile"
                },
                "filter_name": {
                    "type": "string",
                    "description": "Name of the filter to modify (PK from List Filters endpoint)"
                },
                "status": {
                    "type": "integer",
                    "description": "Status to set (1 to enable, 0 to disable)",
                    "enum": [0, 1]
                }
            },
            "required": ["auth_token", "profile_id", "filter_name", "status"]
        }
    }
]

In [89]:
def format_profile_response(response: Dict) -> str:
    """Format profile listing response to show useful information"""
    if not response.get('success'):
        return f"Error: {response.get('message', 'Unknown error')}"
        
    profiles = response['body']['profiles']
    result = "\nProfiles:\n" + "-" * 50 + "\n"
    for profile in profiles:
        result += f"Name: {profile['name']}\n"
        result += f"ID: {profile['PK']}\n"
        result += f"Last Updated: {profile['updated']}\n"
        result += "-" * 50 + "\n"
    return result

def format_filter_response(response: Dict) -> str:
    """Format filter listing response to show useful information"""
    if not response.get('success'):
        return f"Error: {response.get('message', 'Unknown error')}"
        
    filters = response['body']['filters']
    result = "\nFilters:\n" + "-" * 50 + "\n"
    for f in filters:
        result += f"Name: {f['name']} (ID: {f['PK']})\n"
        result += f"Status: {'Enabled' if f.get('status', 0) == 1 else 'Disabled'}\n"
        
        # Handle multi-level filters
        if 'levels' in f:
            result += "Levels:\n"
            for level in f['levels']:
                result += f"  - {level['title']}: "
                result += f"{'Enabled' if level.get('status', 0) == 1 else 'Disabled'}\n"
        
        result += "-" * 50 + "\n"
    return result

def format_modify_response(response: Dict, filter_name: str, status: int) -> str:
    """Format filter modification response"""
    if not response.get('success'):
        return f"Error modifying {filter_name}: {response.get('message', 'Unknown error')}"
    
    return f"\nSuccessfully {'enabled' if status == 1 else 'disabled'} filter: {filter_name}"

def handle_tool_response(content_block, auth_token):
    """Helper function to handle different tool responses with improved formatting"""
    try:
        if content_block.name == "create_profile":
            result = create_controld_profile(
                auth_token=auth_token,
                name=content_block.input.get("name"),
                clone_profile_id=content_block.input.get("clone_profile_id")
            )
            if result.get('success'):
                print(f"\nProfile created successfully: {content_block.input.get('name')}")
                if content_block.input.get('clone_profile_id'):
                    print(f"Cloned from profile ID: {content_block.input.get('clone_profile_id')}")
            else:
                print(f"\nError creating profile: {result.get('message', 'Unknown error')}")
        
        elif content_block.name == "get_profiles":
            result = get_controld_profiles(auth_token)
            print(format_profile_response(result))
        
        elif content_block.name == "get_filters":
            result = get_controld_filters(
                auth_token=auth_token,
                profile_id=content_block.input.get("profile_id")
            )
            print(format_filter_response(result))
        
        elif content_block.name == "modify_filter":
            result = modify_controld_filter(
                auth_token=auth_token,
                profile_id=content_block.input.get("profile_id"),
                filter_name=content_block.input.get("filter_name"),
                status=content_block.input.get("status")
            )
            print(format_modify_response(
                result,
                content_block.input.get("filter_name"),
                content_block.input.get("status")
            ))
            
    except Exception as e:
        print(f"\nError executing {content_block.name}: {str(e)}")
        import traceback
        print(traceback.format_exc())

def chat_with_controld(user_query: str, auth_token: str):
    messages = [{
        "role": "user", 
        "content": f"Auth Token: {auth_token}\n\nUser Query: {user_query}"
    }]
    
    system_prompt = """
    You are an assistant that helps users interact with their ControlD profiles.
    You have access to the following tools:
    1. get_profiles - Lists all existing profiles
    2. create_profile - Creates a new profile or clones an existing one
    3. get_filters - Lists all native filters for a specific profile
    4. modify_filter - Enables or disables a filter for a specific profile
    
    When users want to modify a filter:
    1. First use get_profiles to get the profile ID if needed
    2. Then use get_filters to confirm the filter exists and understand its structure
    3. Finally use modify_filter with the correct filter name and status
    
    Important notes for filter modification:
    - For filters with multiple levels (like 'ads' with Relaxed/Balanced/Strict),
      use the specific level name (e.g., 'ads_small', 'ads_medium', 'ads')
    - Status should be 1 to enable and 0 to disable
    - After modifying a filter, verify the change by getting the filters again
    
    Do not just describe what you'll do - actually make the API calls using the tools.
    """
    
    # Get initial response
    response = client.messages.create(
        system=system_prompt,
        model=MODEL_NAME,
        max_tokens=1000,
        messages=messages,
        tools=tools,
    )
    
    last_content_block = response.content[-1]
    if last_content_block.type == "tool_use":
        try:
            if last_content_block.name == "get_profiles":
                profiles = get_controld_profiles(auth_token)
                print(format_profile_response(profiles))
                
                create_messages = [
                    *messages,
                    {"role": "assistant", "content": "Retrieved profile information."},
                    {"role": "user", "content": f"Profiles data: {profiles}"}
                ]
                
                create_response = client.messages.create(
                    system=system_prompt,
                    model=MODEL_NAME,
                    max_tokens=1000,
                    messages=create_messages,
                    tools=tools,
                )
                
                next_block = create_response.content[-1]
                if next_block.type == "tool_use":
                    handle_tool_response(next_block, auth_token)
                    
                    # If we modified a filter, verify the change
                    if next_block.name == "modify_filter":
                        verify_messages = [
                            *create_messages,
                            {"role": "assistant", "content": "Verifying filter modification..."},
                        ]
                        
                        verify_response = client.messages.create(
                            system=system_prompt,
                            model=MODEL_NAME,
                            max_tokens=1000,
                            messages=verify_messages,
                            tools=tools,
                            tool_choice={"type": "tool", "name": "get_filters"}
                        )
                        
                        verify_block = verify_response.content[-1]
                        if verify_block.type == "tool_use":
                            handle_tool_response(verify_block, auth_token)
            
            else:
                handle_tool_response(last_content_block, auth_token)
                
        except Exception as e:
            print(f"\nError in chat handler: {str(e)}")
            import traceback
            print(traceback.format_exc())

In [90]:
# Get auth token
controld_token = os.getenv("CONTROLD_API_TOKEN")
if not controld_token:
    raise ValueError("Please set CONTROLD_API_TOKEN environment variable")

# Test queries
queries = [
    "Disable the social media filter for the 'arun' profile",
]

for query in queries:
    chat_with_controld(query, controld_token)


Profiles:
--------------------------------------------------
Name: arun
ID: 626071rduv3k
Last Updated: 1730614582
--------------------------------------------------
Name: family
ID: 626842sjcqro
Last Updated: 1729986987
--------------------------------------------------
Name: relatives
ID: 628525otprmu
Last Updated: 1729134044
--------------------------------------------------


Filters:
--------------------------------------------------
Name: Ads & Trackers (ID: ads)
Status: Enabled
Levels:
  - Relaxed: Disabled
  - Balanced: Disabled
  - Strict: Enabled
--------------------------------------------------
Name: Adult Content (ID: porn)
Status: Enabled
Levels:
  - Relaxed: Disabled
  - Strict: Enabled
--------------------------------------------------
Name: Artificial intelligence (ID: noai)
Status: Disabled
--------------------------------------------------
Name: Clickbait (ID: fakenews)
Status: Enabled
--------------------------------------------------
Name: Crypto (ID: cryptominers)